In [1]:
# !pip install ydata-profiling

import wandb
import pandas as pd
from ydata_profiling import ProfileReport

In [2]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)  # we use save_code=True in the call to wandb.init so the notebook is uploaded and versioned by W&B.

# Fetch data artifact from W&B and read it
local_path = wandb.use_artifact("sample.csv:latest").file()
print(local_path)

wandb: Currently logged in as: adisongoh (adisongoh-national-university-of-singapore). Use `wandb login --relogin` to force relogin


./artifacts/sample.csv:latest/sample1.csv


In [3]:
df = pd.read_csv(local_path)
print(df.shape)

(20000, 16)


In [8]:
profile = ProfileReport(df, title="Profiling Report")

print("Profile ready")

Profile ready


In [10]:
# Export the report to a html file
profile.to_file("profile_report_EDA_before.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|███████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 57.15it/s]
/home/adisongoh/miniconda3/envs/mlflow-e4fcefd643ef1ee947cfeff48521954cf08a16aa/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:87: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'putmask: first argument must be an array')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

What do you notice? Look around and see what you can find. 

- For example, there are missing values in a few columns and the column last_review is a date but it is in string format.
- Look also at the price column, and note the outliers. There are some zeros and some very high prices. After talking to your stakeholders, you decide to consider from a minimum of 10 to a maximum of 350 per night.

In [19]:
# Drop outliers - price between 10 to 350
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

# minimum_nights between 1 to 14
idx = df['minimum_nights'].between(1, 14)
df = df[idx].copy()

# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

Note how we did not impute missing values. We will do that in the inference pipeline, so we will be able to handle missing values also in production.



In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16747 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              16747 non-null  int64         
 1   name                            16740 non-null  object        
 2   host_id                         16747 non-null  int64         
 3   host_name                       16740 non-null  object        
 4   neighbourhood_group             16747 non-null  object        
 5   neighbourhood                   16747 non-null  object        
 6   latitude                        16747 non-null  float64       
 7   longitude                       16747 non-null  float64       
 8   room_type                       16747 non-null  object        
 9   price                           16747 non-null  int64         
 10  minimum_nights                  16747 non-null  int64         
 11  number_

In [21]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
count,1.674700e+04,1.674700e+04,16747.000000,16747.000000,16747.000000,16747.000000,16747.000000,13842,13842.000000,16747.000000,16747.000000
mean,1.873316e+07,6.600128e+07,40.727028,-73.949359,121.082642,2.860572,25.856392,2018-10-04 03:07:21.612483584,1.469779,3.105153,97.520571
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,10.000000,1.000000,0.000000,2011-05-12 00:00:00,0.010000,1.000000,0.000000
25%,9.287530e+06,7.748101e+06,40.687595,-73.981230,67.000000,1.000000,1.000000,2018-07-01 00:00:00,0.210000,1.000000,0.000000
50%,1.931783e+07,3.052348e+07,40.720070,-73.953650,100.000000,2.000000,7.000000,2019-05-26 00:00:00,0.830000,1.000000,25.000000
75%,2.861656e+07,1.019804e+08,40.762980,-73.932665,159.000000,3.000000,27.000000,2019-06-24 00:00:00,2.180000,2.000000,179.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,350.000000,14.000000,607.000000,2019-07-08 00:00:00,27.950000,327.000000,365.000000
std,1.090390e+07,7.770122e+07,0.056214,0.047920,70.664779,2.291717,47.305260,NaN,1.734220,20.105021,124.838908


In [22]:
profile2 = ProfileReport(df, title="Profiling Report")

print("Profile ready")

Profile ready


In [24]:
# Export the report to a html file
profile2.to_file("profile_report_EDA_after.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|███████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00,  8.29it/s]
/home/adisongoh/miniconda3/envs/mlflow-e4fcefd643ef1ee947cfeff48521954cf08a16aa/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:87: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'putmask: first argument must be an array')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
# Terminate the run
run.finish()